
<div dir=rtl >
<h1>قوانین انجمنی</h1>

قوانین انجمنی روابط و وابستگی‌های متقابل بین مجموعه بزرگی از اقلام داده‌ای را نشان می‌دهند. 

مثال متداول در رابطه با کشف قوانین انجمنی "تحلیل سبد خرید" است.  در این فرایند با توجه به اقلام مختلفی که مشتریان در سبد خریدشان قرار می‌دهند، عادات و رفتار خرید مشتریان مورد تحلیل قرار می‌گیرد و می‌توان با شناسایی ارتباط بین محصولات، الگوهای تکرار شونده در حین خرید را بدست آورد.

سه پارامتر مهم:
- Support  میزان محبوبیت یک مجموعه آیتم را با توجه به دفعات حضور در تراکنش‌ها نشان می‌دهد.
- Confidence  احتمال خرید آیتم y در صورت خرید آیتم x را نشان می‌دهد. x -> y
- Lift   ترکیب دو پارامتر بالا می‌باشد.

برای پیاده سازی قوانین انجمنی در این تمرین، از الگوریتم Apriori که یکی از محبوب‌ترین و کارآمدترین الگوریتم‌های این حوزه است استفاده می‌کنیم.


<font color='Green'> پرسش:  تاثیر مقادیر مختلف پارامتر Lift را در احتمال وقوع نتیجه بررسی کنید.  </font>


</div>





<div dir=rtl >
<h1>الگوریتم Apriori</h1>

روش کار الگوریتم به این صورت است که یک  حداقل مقدار پشتیبانی در نظر گرفته    می‌شود و تکرارها با مجموعه آیتم‌های مکرر اتفاق می‌افتند. در صورتیکه  مجموعه‌ها و زیرمجموعه‌ها مقدار پشتیبانی کمتر از حد آستانه داشته باشند حذف می‌شوند. این روند تا زمانیکه امکان حذف وجود نداشته باشد ادامه می‌یابد. 

در این بخش از تمرین می‌خواهیم الگوریتم apriori را بر روی مجموعه داده Hypermarket_dataset که شامل سفارشات خرید افراد از فروشگاه های مواد غذایی است اعمال کنیم.


</div>





<div dir=rtl >
<h1>آماده‌سازی داده</h1>

<font color='Green'>پرسش: برای شروع کار باید داده‌های دیتاست را در قالب ماتریس پراکنده که محصولات  خریداری شده در ستون‌ و شماره سفارشات خرید به عنوان شاخص (index) هستند آماده کنید. 


برای راحتی کار محصولات خریداری شده در هر سفارش را با اعداد 0,1 کدگذاری کنید.

 نمونه ماتریس خروجی شما:

<img src="https://drive.google.com/uc?id=1eD0jan1ZbeYqSklgK--ks7oeY-MyTA3p"></img>

</div>




In [41]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import tqdm
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [15]:
df = pd.read_csv("Hypermarket_dataset.csv")
df = df.sort_values(by=['Member_number', 'Date'])
df.head()

,Member_number,Date,itemDescription
4843,1000,15-03-2015,sausage
8395,1000,15-03-2015,whole milk
20992,1000,15-03-2015,semi-finished bread
24544,1000,15-03-2015,yogurt
13331,1000,24-06-2014,whole milk


In [106]:
items = df["itemDescription"].unique()
groups = df.groupby(by=["Member_number", "Date"])


In [107]:
rows = []
for g in groups:
    rows.append([*[g[0]], *[(item in list(g[1]["itemDescription"])) for item in items]])
    


In [108]:
matrix = pd.DataFrame(rows, columns=[*["order"], *items]).set_index("order")
matrix

,sausage,whole milk,semi-finished bread,yogurt,pastry,salty snack,canned beer,misc. beverages,hygiene articles,soda,...,nuts/prunes,soap,decalcifier,kitchen utensil,organic products,frozen chicken,salad dressing,specialty vegetables,toilet cleaner,rubbing alcohol
order,,,,,,,,,,,,,,,,,,,,,
"(1000, 15-03-2015)",True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(1000, 24-06-2014)",False,True,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(1000, 24-07-2015)",False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
"(1000, 25-11-2015)",True,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
"(1000, 27-05-2015)",False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(4999, 24-01-2015)",False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(4999, 26-12-2015)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(5000, 09-03-2014)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False



<div dir=rtl >
<h1>شناسایی الگوهای پر تکرار</h1>

<font color='Green'> پرسش:  با اعمال الگوریتم apriori و به ازای مقدار min_support = 0.07 تمامی الگوهای پر تکرار را تولید نمایید. 

</div>




In [109]:
frequents = apriori(matrix, min_support=0.07, use_colnames=True)
frequents

,support,itemsets
0,0.157923,(whole milk)
1,0.085879,(yogurt)
2,0.097106,(soda)
3,0.110005,(rolls/buns)
4,0.122101,(other vegetables)



<div dir=rtl >
<h1>استخراج قوانین انجمنی</h1>


<font color='Green'> پرسش: تابعی بنویسید که با گرفتن دو ورودی confidence و lift قوانین انجمنی حاصل را در خروجی نشان دهد. </br>
برای دو حالت خروجی خود را در گزارش ثبت کنید.


</div>




In [110]:
def ass_rules(matrix, lift, confidence):
    rules = association_rules(matrix, metric="lift", min_threshold=lift)
    return rules[rules['confidence'] >= confidence]

In [111]:
ass_rules(frequents, 1, 0.4)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
